In [ ]:
#|default_exp ffcv.transforms

In [ ]:
#|export
from __future__ import annotations

import math
import numpy as np
from numpy.random import rand
from typing import Callable, Optional, Tuple
from dataclasses import replace

from ffcv.pipeline.compiler import Compiler
from ffcv.pipeline.allocation_query import AllocationQuery
from ffcv.pipeline.operation import Operation
from ffcv.pipeline.state import State

from ffcv.transforms.cutout import Cutout
from ffcv.transforms.flip import RandomHorizontalFlip as _RandomHorizontalFlip
from ffcv.transforms.random_resized_crop import RandomResizedCrop
from ffcv.transforms.translate import RandomTranslate
from ffcv.transforms.color_jitter import RandomBrightness as _RandomBrightness
from ffcv.transforms.color_jitter import RandomContrast as _RandomContrast
from ffcv.transforms.color_jitter import RandomSaturation as _RandomSaturation
from ffcv.transforms.poisoning import Poison
from ffcv.transforms.replace_label import ReplaceLabel
from ffcv.transforms.common import Squeeze

In [ ]:
#|export
_all_ = ['RandomResizedCrop', 'RandomTranslate', 'Cutout', 'Poison', 'ReplaceLabel', 'Squeeze']

In [ ]:
#|hide
from nbdev.showdoc import show_doc

# FFCV Transforms
> Image transformations from FFCV

Also contains transforms not yet merged into FFCV. FFCV Transforms are a doc reference for transforms provided by FFCV.

## FFCV Transforms

The FFCV Transforms here have had thier initilization arguments harminized, and are otherwise identical to their FFCV counterparts.

In [ ]:
show_doc(RandomResizedCrop)

In [ ]:
#|export
class RandomHorizontalFlip(_RandomHorizontalFlip):
    """
    Flip the image horizontally with probability flip_prob.
    Operates on raw arrays (not tensors).

    Parameters
    ----------
    prob : float
        The probability with which to flip each image in the batch
        horizontally.
    """

    def __init__(self, prob: float = 0.5):
        super().__init__(prob)

In [ ]:
show_doc(Cutout)

In [ ]:
show_doc(RandomTranslate)

In [ ]:
#|export
class RandomBrightness(_RandomBrightness):
    '''
    Randomly adjust image brightness. Operates on raw arrays (not tensors).

    Parameters
    ----------
    prob : float
        probability to apply brightness
    magnitude : float
        randomly choose brightness enhancement factor on [max(0, 1-magnitude), 1+magnitude]
    '''
    def __init__(self, prob:float, magnitude: float):
        super().__init__(magnitude, prob)

In [ ]:
#|export
class RandomContrast(_RandomContrast):
    '''
    Randomly adjust image contrast. Operates on raw arrays (not tensors).

    Parameters
    ----------
    prob : float
        probability to apply contrast
    magnitude : float
        randomly choose contrast enhancement factor on [max(0, 1-magnitude), 1+magnitude]
    '''
    def __init__(self, prob:float, magnitude:float):
        super().__init__(magnitude, prob)

In [ ]:
#|export
class RandomSaturation(Operation):
    '''
    Randomly adjust image color balance. Operates on raw arrays (not tensors).

    Parameters
    ----------
    prob : float
        probability to apply saturation
    magnitude : float
        randomly choose color balance enhancement factor on [max(0, 1-magnitude), 1+magnitude]
    '''
    def __init__(self, prob:float, magnitude:float):
        super().__init__(magnitude, prob)

In [ ]:
show_doc(Poison)

In [ ]:
show_doc(ReplaceLabel)

In [ ]:
show_doc(Squeeze)

## Additional Transforms

These transforms are pending acceptance and/or release for the next version of FFCV and will be converted to a documentation reference.

In [ ]:
#|export
class RandomCutout(Cutout):
    """Random cutout data augmentation (https://arxiv.org/abs/1708.04552).

    Parameters
    ----------
    prob : float
        Probability of applying on each image.
    crop_size : int
        Size of the random square to cut out.
    fill : Tuple[int, int, int], optional
        An RGB color ((0, 0, 0) by default) to fill the cutout square with.
        Useful for when a normalization layer follows cutout, in which case
        you can set the fill such that the square is zero post-normalization.
    """
    def __init__(self, prob: float, crop_size: int, fill: Tuple[int, int, int] = (0, 0, 0)):
        super().__init__(crop_size, fill)
        self.prob = np.clip(prob, 0., 1.)

    def generate_code(self) -> Callable:
        my_range = Compiler.get_iterator()
        crop_size = self.crop_size
        fill = self.fill
        prob = self.prob

        def cutout_square(images, *_):
            should_cutout = rand(images.shape[0]) < prob
            for i in my_range(images.shape[0]):
                if should_cutout[i]:
                    # Generate random origin
                    coord = (
                        np.random.randint(images.shape[1] - crop_size + 1),
                        np.random.randint(images.shape[2] - crop_size + 1),
                    )
                    # Black out image in-place
                    images[i, coord[0]:coord[0] + crop_size, coord[1]:coord[1] + crop_size] = fill
            return images

        cutout_square.is_parallel = True
        return cutout_square

In [ ]:
#|export

# Implementation inspired by fastai https://docs.fast.ai/vision.augment.html#randomerasing
# fastai - Apache License 2.0 - Copyright (c) 2023 fast.ai
class RandomErasing(Operation):
    """Random erasing data augmentation (https://arxiv.org/abs/1708.04896).

    Parameters
    ----------
    prob : float
        Probability of applying on each image.
    min_area : float
        Minimum erased area as percentage of image size.
    max_area : float
        Maximum erased area as percentage of image size.
    min_aspect : float
        Minimum aspect ratio of erased area.
    max_count : int
        Maximum number of erased blocks per image. Erased Area is scaled by max_count.
    fill_mean : Tuple[int, int, int], optional
        The RGB color mean (ImageNet's (124, 116, 103) by default) to randomly fill the
        erased area with. Should be the mean of dataset or pretrained dataset.
    fill_std : Tuple[int, int, int], optional
        The RGB color standard deviation (ImageNet's (58, 57, 57) by default) to randomly
        fill the erased area with. Should be the st. dev of dataset or pretrained dataset.
    fast_fill : bool
        Default of True is ~2X faster by generating noise once per batch and randomly
        selecting slices of the noise instead of generating unique noise per each image.
    """
    def __init__(self, prob: float, min_area: float = 0.02, max_area: float = 0.3,
                 min_aspect: float = 0.3, max_count: int = 1,
                 fill_mean: Tuple[int, int, int] = (124, 116, 103),
                 fill_std: Tuple[int, int, int] = (58, 57, 57),
                 fast_fill : bool = True):
        super().__init__()
        self.prob = np.clip(prob, 0., 1.)
        self.min_area = np.clip(min_area, 0., 1.)
        self.max_area = np.clip(max_area, 0., 1.)
        self.log_ratio = (math.log(np.clip(min_aspect, 0., 1.)), math.log(1/np.clip(min_aspect, 0., 1.)))
        self.max_count = max_count
        self.fill_mean = np.array(fill_mean)
        self.fill_std = np.array(fill_std)
        self.fast_fill = fast_fill

    def generate_code(self) -> Callable:
        my_range = Compiler.get_iterator()
        prob = self.prob
        min_area = self.min_area
        max_area = self.max_area
        log_ratio = self.log_ratio
        max_count = self.max_count
        fill_mean = self.fill_mean
        fill_std = self.fill_std
        fast_fill = self.fast_fill

        def random_erase(images, *_):
            if fast_fill:
                noise = fill_mean + (fill_std * np.random.randn(images.shape[1], images.shape[2], images.shape[3])).astype(images.dtype)

            should_cutout = rand(images.shape[0]) < prob
            for i in my_range(images.shape[0]):
                if should_cutout[i]:
                    count = np.random.randint(1, max_count) if max_count > 1 else 1
                    for j in range(count):
                        # Randomly select bounds
                        area = np.random.uniform(min_area, max_area, 1) * images.shape[1] * images.shape[2] / count
                        aspect = np.exp(np.random.uniform(log_ratio[0], log_ratio[1], 1))
                        bound = (
                            int(round(np.sqrt(area * aspect).item())),
                            int(round(np.sqrt(area / aspect).item())),
                        )
                        # Select random erased area
                        coord = (
                            np.random.randint(0, max(1, images.shape[1] - bound[0])),
                            np.random.randint(0, max(1, images.shape[2] - bound[1])),
                        )
                        # Fill image with random noise in-place
                        if fast_fill:
                            images[i, coord[0]:coord[0] + bound[0], coord[1]:coord[1] + bound[1]] =\
                                noise[coord[0]:coord[0] + bound[0], coord[1]:coord[1] + bound[1]]
                        else:
                            noise = fill_mean + (fill_std * np.random.randn(bound[0], bound[1], images.shape[3])).astype(images.dtype)
                            images[i, coord[0]:coord[0] + bound[0], coord[1]:coord[1] + bound[1]] = noise
            return images

        random_erase.is_parallel = True
        return random_erase

    def declare_state_and_memory(self, previous_state: State) -> Tuple[State, Optional[AllocationQuery]]:
        return replace(previous_state, jit_mode=True), None

In [ ]:
#|export
# RandomHue adapted from pending FFCV PR: https://github.com/libffcv/ffcv/pull/226
# FFCV - Apache License 2.0 - Copyright (c) 2022 FFCV Team

# Code for Hue adapted from:
# https://sanje2v.wordpress.com/2021/01/11/accelerating-data-transforms/
# https://stackoverflow.com/questions/8507885

class RandomHue(Operation):
    '''
    Randomly adjust image hue Operates on raw arrays (not tensors).

    Parameters
    ----------
    prob : float
        probability to apply hue shift
    magnitude : float
        Randomly choose hue factor on [-magnitude, magnitude], clipped to [-0.5, 0.5].
        0.5 and -0.5 give a complete reversal of the hue channel. 0 means no shift.
    '''
    def __init__(self, prob:float, magnitude:float):
        super().__init__()
        self.prob = prob
        self.magnitude  = np.clip(magnitude, 0, 0.5)

    def generate_code(self):
        my_range = Compiler.get_iterator()
        prob = self.prob
        magnitude = self.magnitude

        def hue(images, dst):
            apply_hue = np.random.rand(images.shape[0]) < prob
            magnitudes = np.random.uniform(-magnitude, magnitude, images.shape[0])
            for i in my_range(images.shape[0]):
                if apply_hue[i] and magnitudes[i]!=0.:
                    img = images[i] / 255.0
                    hue_factor_radians = magnitudes[i] * 2.0 * np.pi
                    cosA = np.cos(hue_factor_radians)
                    sinA = np.sin(hue_factor_radians)
                    hue_rotation_matrix =\
                        [[cosA + (1.0 - cosA) / 3.0, 1./3. * (1.0 - cosA) - np.sqrt(1./3.) * sinA, 1./3. * (1.0 - cosA) + np.sqrt(1./3.) * sinA],
                        [1./3. * (1.0 - cosA) + np.sqrt(1./3.) * sinA, cosA + 1./3.*(1.0 - cosA), 1./3. * (1.0 - cosA) - np.sqrt(1./3.) * sinA],
                        [1./3. * (1.0 - cosA) - np.sqrt(1./3.) * sinA, 1./3. * (1.0 - cosA) + np.sqrt(1./3.) * sinA, cosA + 1./3. * (1.0 - cosA)]]
                    hue_rotation_matrix = np.array(hue_rotation_matrix, dtype=img.dtype)

                    for row in range(img.shape[0]):
                        for col in range(img.shape[1]):
                            r, g, b = img[row, col, :]
                            img[row, col, 0] = r * hue_rotation_matrix[0, 0] + g * hue_rotation_matrix[0, 1] + b * hue_rotation_matrix[0, 2]
                            img[row, col, 1] = r * hue_rotation_matrix[1, 0] + g * hue_rotation_matrix[1, 1] + b * hue_rotation_matrix[1, 2]
                            img[row, col, 2] = r * hue_rotation_matrix[2, 0] + g * hue_rotation_matrix[2, 1] + b * hue_rotation_matrix[2, 2]
                    dst[i] = np.asarray(np.clip(img * 255., 0, 255), dtype=np.uint8)
                else:
                    dst[i] = images[i]
            return dst

        hue.is_parallel = True
        return hue

    def declare_state_and_memory(self, previous_state: State) -> Tuple[State, Optional[AllocationQuery]]:
        return (replace(previous_state, jit_mode=True), AllocationQuery(shape=previous_state.shape, dtype=previous_state.dtype))